In [1]:
import pybedtools
import pandas as pd
import os
import sys

sys.path.append("/ifs3/clliu/tools/")
from utils import ensure_directory_exists,aggregate_dataframe

In [12]:
samplefile = "/ifs3/scdata/RNA-Seq_counts/sample"
samplefilepath = pd.read_csv(samplefile,header=None)

In [40]:
transcript_filepath = "/ifs3/scdata/GRCh38gtf/Homo_sapiens.GRCh38.106.modif.bed"
a = pybedtools.BedTool(transcript_filepath)
for i in range(len(samplefilepath)):
    input = samplefilepath[0][i]+"/gene_abund.tab"
    outdir = samplefilepath[0][i]+"/transcript_anno.bed"


    data = pd.read_csv(input,sep = '\t')
    data = data.drop(['Gene ID','Gene Name','Coverage','FPKM','TPM'],axis=1)
    data  = data[['Reference','Start','End','Strand']]
    data.to_csv(samplefilepath[0][i]+'/gene_abund.bed',sep='\t',header=None,index=None)
    # overlap
    b = pybedtools.BedTool(samplefilepath[0][i]+'/gene_abund.bed')
    intersected = b.intersect(a,wa=True,wb=True)
    intersected.saveas(outdir)
    df = pd.read_csv("/ifs3/scdata/RNA-Seq_counts/RS1807-011-T_out/transcript_anno.bed",sep='\t',header=None)
    df = df.drop([4,7,8,10],axis=1)
    df['room'] = df[0] + ':' + df[1].astype(str) + '-' + df[2].astype(str)

    duplicates = df.duplicated('room', keep='first')
    result_df = df[~duplicates]
    result_df = result_df.drop('room',axis=1)
    result_df = result_df[[0,5,6,1,2,9,3,11,12]]
    result_df.to_csv(samplefilepath[0][i]+"/transcript_anno_duplicates.bed",sep='\t',header=None,index=None)